In [2]:
import pandas as pd
data = pd.read_csv("comments.tsv", sep='\t')
data

,should_ban,comment_text
0,0,The picture on the article is not of the actor...
1,1,"Its madness. Shes of Chinese heritage, but JAP..."
2,1,Fuck You. Why don't you suck a turd out of my ...
3,1,God is dead\r\nI don't mean to startle anyone ...
4,1,THIS USER IS A PLANT FROM BRUCE PERENS AND GRO...
...,...,...
995,0,rowspan=9 colspan=8|Did Not Qualify
996,0,"""== Disputed and under-referenced ==\r\n\r\nI ..."
997,0,Why?\r\nWhy does this event have its own page?...
998,0,"Que? \r\n\r\nWas this fat fingers? If not, ca..."


In [25]:
from nltk.tokenize import TweetTokenizer
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
tokenizer = TweetTokenizer()
def preprocess(text):
    words = tokenizer.tokenize(text)
    for i in range(len(words)):
        words[i] = morph.parse(words[i])[0].normal_form
    return ' '.join(words).lower()
words = set(" ".join(list(data.comment_text)).split())

In [26]:
data["comment_text"] = data["comment_text"].apply(preprocess)

In [27]:
X = data["comment_text"]
Y = data["should_ban"]

In [58]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=3)

In [59]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(X_train)
X_train = count_vect.transform(X_train)
X_test = count_vect.transform(X_test)

In [61]:
from sklearn.metrics import accuracy_score

In [62]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

models = {
    'tree': DecisionTreeClassifier(max_depth=10),
    'forest': RandomForestClassifier(n_jobs=-1),
    'boosting': GradientBoostingClassifier(),
    'logistic': LogisticRegression(),
    'neigh': KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
}

for i in models:
    print('\n' + i)
    models[i].fit(X_train,y_train)
    y_pred_train = models[i].predict(X_train)
    y_pred_test = models[i].predict(X_test)
#     print(accuracy_score(y_train,y_pred_train)) 
#     print(accuracy_score(y_test,y_pred_test))
    print(roc_auc_score(y_train,y_pred_train)) 
    print(roc_auc_score(y_test,y_pred_test))


tree
0.8464811399354834
0.6938225671948168

forest
1.0
0.730450959009558

boosting
0.9131518418560268
0.7942138687536083

logistic
1.0
0.7900121880813396

neigh
0.8249186421608072
0.6701520302777599


**TF/IDF with stop words**   
tree
0.88783299745779
0.7021617807428314

forest
1.0
0.7052408749759446

boosting
0.938769769776898
0.8117582911027007

logistic
0.9385810623161882
0.7999230226441723

neigh
0.8723198199802036
0.7464558342420938


**COUNT VECTORIZER with stop words**  
tree
0.8493333333333334
0.696

forest
1.0
0.692

boosting
0.9146666666666666
0.792

logistic
1.0
0.788

neigh
0.828
0.664

**COUNT VECTORIZER without stop words**  
tree
0.7386666666666667
0.6

forest
1.0
0.76

boosting
0.9066666666666666
0.72

logistic
0.9986666666666667
0.764

neigh
0.7453333333333333
0.66

**TF/IDF without stop_words**  
tree
0.7219057317220803
0.6045288344345372

forest
1.0
0.7700942972608891

boosting
0.9356009086442258
0.7346526396818269

logistic
0.986747751532803
0.7942138687536083

neigh
0.8650385604113111
0.7101481814099685

**Лучше всего TF/IDF без исключения стоп слов**